### requests의  PrepardRequest
* [PrepardRequest](https://requests.readthedocs.io/en/latest/user/advanced/#prepared-requests)

In [ ]:
from requests import Request, Session

NAVER_CLIENT_ID = 'ZAZjlfXS4P5FW5DtE0wg'
NAVER_CLIENT_SECRET = 'RNbe1ikysg'


def get_text_list():
    lists = []
    with open('data/yesterday.txt', 'r',encoding="utf8") as file:
        contents = file.read()
        lists = contents.split("\n")

    return lists

def save_to_file(list):
    with open('data/yesterday.translated_ver2.txt', 'w',encoding="utf8") as file:
        file.writelines(list)

def main():
    # session 객체생성
    session = Session()
    # 요청헤더 dict 선언
    headers = {
        'X-Naver-Client-Id': NAVER_CLIENT_ID,
        'X-Naver-Client-Secret': NAVER_CLIENT_SECRET,
    }
    url = 'https://openapi.naver.com/v1/papago/n2mt'

    lists = get_text_list()
    lists = [ i for i in lists if len(i) != 0] # '' 문자열 제거

    result_list = []
    payload = {
        'source': 'en',
        'target': 'ko',
    }
    for en_text in lists:
        payload['text'] = en_text
        # request 객체생성
        req = Request('POST', url, data=payload, headers=headers)
        # PreparedRequest 객체생성
        prepared = req.prepare()
        #print(type(prepared))
        # request를 보내고 response 객체생성
        res = session.send(prepared)
        
        print('send original text : ', en_text)
        ko_text = ''
        
        try:
            ko_text = res.json()['message']['result']['translatedText']
        except Exception as err:
            print('에러 발생 ',res.json())
            break

        print('translated to : ', ko_text)
        result_list.append(en_text+ '\n')
        result_list.append(ko_text + '\n')

    save_to_file(result_list)
    print('job completed..')


main()


#### googletrans 라이브러리 사용하여 번역하기

In [ ]:
# -q 옵션은 install 한 결과를 출력하지 않음
%pip install -q googletrans

In [ ]:
%pip show googletrans

In [ ]:
from googletrans import Translator

def get_text_list():
    lists = []
    with open('data/yesterday.txt', 'r', encoding="utf8") as file:
        contents = file.read()
        lists = contents.split("\n")
    return lists


def save_to_file(list):
    with open('data/yesterday_translated_google.txt', 'w', encoding="utf8") as file:
        # list를 file로 저장
        file.writelines(list)

def main():
    # Translator 객체생성
    translator = Translator()

    # txt 파일을 읽어서 list로 저장하기
    lists = get_text_list()
    # empty string 제거
    lists = [i for i in lists if len(i.strip()) != 0]  # 공백 제거

    result_list = []

    for en_text in lists:
        print('원문:', en_text)
        # 번역 요청청
        translated = translator.translate(en_text, src='en', dest='ko')
        ko_text = translated.text
        print('번역:', ko_text)

        result_list.append(en_text + '\n')
        result_list.append(ko_text + '\n')

    save_to_file(result_list)
    print('번역 완료!')

main()